# Webscraping code for Business Times
Website: https://www.businesstimes.com.sg/keywords/straits-times-index
- Keyword: Straits Times Index


### Import libraries

In [1]:
#!pip install webdriver-manager
from bs4 import BeautifulSoup
from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

import csv
import numpy as np
import pandas as pd
from random import randint
import time
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

In [2]:
print("Last update:",datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Last update: 12/03/2022 19:22:51


### Scrape information
To download adblocker .crx extension: https://www.crx4chrome.com/crx/31927/

In [3]:
# # Access URL
# bt_url = 'https://www.businesstimes.com.sg/keywords/straits-times-index'

# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.implicitly_wait(20)
# driver.get(bt_url)

# Access URL and Handle Advertisement Pop-up
bt_url = 'https://www.businesstimes.com.sg/keywords/straits-times-index'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_extension("/Users/tohjiaxuan/Library/Application Support/Google/Chrome/Default/Extensions/gighmmpiobklfepjocnamgkkbiglidom-4.44.0-Crx4Chrome.com.crx")

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
driver.implicitly_wait(20)
driver.get(bt_url)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/tohjiaxuan/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [4]:
# 2 Tabs will open - The Business Times and AdBlocker Installation > Switch to Tab 1
time.sleep(30)
driver.switch_to.window(driver.window_handles[0])

In [5]:
# Say 'No thanks' to subscription pop-up if it appears
try:
    subscription_button = driver.find_element_by_xpath('//*[@id="ei_subscribe"]/div/div/div/div[2]/span')
    if subscription_button:
        subscription_button.click()
except ElementNotInteractableException as e1:
    print('Nothing to press')
except NoSuchElementException as e2:
    print('Nothing to press')

Nothing to press


In [6]:
# //*[@id="block-system-main"]/div/div[2]/div/div[3]/div/div/ul/li/a

In [7]:
# Press 'MORE STORIES' until the end to get all possible articles
more_stories = True
while more_stories:
    try:
        find_button = driver.find_element_by_xpath('//*[@id="block-system-main"]/div/div[2]/div/div[3]/div/div/ul/li/a')
        if find_button != None:
            find_button.click()
            time.sleep(3)
        else:
            more_stories = False
    except StaleElementReferenceException as e1:
        continue
    except ElementClickInterceptedException as e2:
        continue
    except NoSuchElementException as e3:
        print('Page fully loaded')
        break

Page fully loaded


In [8]:
# Scrape information
page_soup = BeautifulSoup(driver.page_source)
information_block = page_soup.find('div',{'class':'col-lg-8 col-md-8 col-sm-12 col-xs-12 hidden--widget-action classy-panel-styles region'})

# Initialisations
news_header = []
news_description = []
news_date = []
url = []

for header in information_block.findAll('h2',{'class':'widget__title'}):
    header_text = header.find('a').text
    news_header.append(header_text)
    
for desc in information_block.findAll('div',{'class':'widget__description'}):
    desc_text = desc.find('p').text
    news_description.append(desc_text)
    
for date in information_block.findAll('div',{'class':'widget__date updated'}):
    news_date.append(date.text)
    
for header in information_block.findAll('h2',{'class':'widget__title'}):
    href = header.find('a').get('href')
    href = 'https://www.businesstimes.com.sg' + href
    url.append(href)

In [9]:
driver.quit()

In [10]:
print(len(news_header))
print(len(news_description))
print(len(news_date))
print(len(url))

504
504
504
504


In [11]:
print(news_header[0])
print(news_description[0])
print(news_date[0])
print(url[0])

Singapore stocks edge up cautiously on Friday
THE Straits Times Index (STI) overcame weak sentiment earlier in the day to register a third straight winning...

Mar 12, 2022 05:50 AM
https://www.businesstimes.com.sg/stocks/singapore-stocks-edge-up-cautiously-on-friday


### Clean up data

In [12]:
# Clean up News Date information

#test = datetime.strptime('Mar 8, 2022 06:00 PM', '%b %d, %Y %I:%M %p')
#test.strftime('%d/%m/%Y')

cleaned_time = [date.strip('\n') for date in news_date]
cleaned_time = [datetime.strptime(date, '%b %d, %Y %I:%M %p') for date in cleaned_time]
cleaned_time = [date.strftime('%d/%m/%Y') for date in cleaned_time]

### Convert to DF and Export to CSV

In [15]:
# Convert to DataFrame
cols = ['Title','Date','Link','Source','Comments']
df = pd.DataFrame({'Title': news_header,
                   'Date': cleaned_time,
                   'Link': url,
                   'Source': 'The Business Times',
                   'Comments': ''}, columns=cols)
df.insert(0, 'Symbol', 'None')
df.head()

,Symbol,Title,Date,Link,Source,Comments
0,None,Singapore stocks edge up cautiously on Friday,12/03/2022,https://www.businesstimes.com.sg/stocks/singap...,The Business Times,
1,None,Singapore stocks edge up cautiously on Friday;...,11/03/2022,https://www.businesstimes.com.sg/stocks/singap...,The Business Times,
2,None,Singapore stocks open lower on Friday; STI dow...,11/03/2022,https://www.businesstimes.com.sg/stocks/singap...,The Business Times,
3,None,Singapore stocks extend gains as Asian markets...,11/03/2022,https://www.businesstimes.com.sg/stocks/singap...,The Business Times,
4,None,Singapore stocks extend gains on Thursday as A...,10/03/2022,https://www.businesstimes.com.sg/stocks/singap...,The Business Times,


In [16]:
# Export to CSV
now = datetime.now()
current = now.strftime("%d/%m/%Y %H:%M:%S")
print('Current date and time',current)

df_dict = df.to_dict(orient='records')
keys = df_dict[0].keys()
with open('business_times_data.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(df_dict)

print('Data for Business Times (keyword "STI") exported successfully!')

Current date and time 12/03/2022 19:30:14
Data for Business Times (keyword "STI") exported successfully!
